In [72]:
import os
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import torchmetrics

import time
import torch.cuda
from flash.image import ImageClassificationData, ImageClassifier
import flash
from pytorch_lightning.loggers import CSVLogger
from timm.loss import LabelSmoothingCrossEntropy
import seaborn as sns
import matplotlib.pyplot as plt
import imageio as iio

In [73]:
DATA_PATH = '../../Pokemon-data/'
NORMALIZED_DATA = '../../Training-baseline/'
SEED = 42

# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 30
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT

In [74]:
metadata = pd.read_csv('metadata/pokemon.csv')

In [75]:
print(metadata.columns)

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')


In [76]:
pokemon_names = [x for x in os.listdir(NORMALIZED_DATA)]

# remove the IDE metafile that was included in the os.listdir
#pokemon_names = pokemon_names[1:]

In [77]:
# get slice from metadata file based on the pokemon used in training
filtered_metadata = metadata[metadata.name.isin(pokemon_names)]

In [78]:
# convert dataframe to list for easier comparison
filtered_list = filtered_metadata.loc[:,'name'].tolist()

In [79]:
# check for differences in training data and the metadata file
print([x for x in pokemon_names if x not in filtered_list])

[]


In [80]:
filtered_metadata = filtered_metadata.loc[:,['name', 'type1']]

In [81]:
print(filtered_metadata.head)

<bound method NDFrame.head of            name   type1
0     Bulbasaur   grass
1       Ivysaur   grass
2      Venusaur   grass
3    Charmander    fire
4    Charmeleon    fire
..          ...     ...
714     Noivern  flying
715     Xerneas   fairy
716     Yveltal    dark
718     Diancie    rock
720   Volcanion    fire

[703 rows x 2 columns]>


In [82]:
def compile_training_data_to_list():
    all_data = []
    for pokemon in os.listdir(NORMALIZED_DATA):
        all_data += [pokemon + '/' + x for x in os.listdir(NORMALIZED_DATA + pokemon)]

    results = create_annotated_dataframe(all_data)
    return results

def create_annotated_dataframe(all_data):
    base_data = {'file_name': [], 'name': [], 'label': []}
    for item in all_data:
        base_data['file_name'].append(item)
        base_data['name'].append(item.split('/')[0])
        # yes, this is a bit ugly, but we have to match with the metadata
        base_data['label'].append(
            filtered_metadata[
                filtered_metadata['name']==(item.split('/')[0])
            ].loc[:,'type1'].tolist()[0])

    results = create_encoded_dataframe(base_data)
    return results

def create_encoded_dataframe(base_data):
    results = pd.DataFrame(base_data, columns = ['file_name', 'name', 'label'])
    le = LabelEncoder()
    labels = le.fit_transform(results['label'])
    results['label'] = np.int64(labels)
    return results

In [83]:
encoded_data = compile_training_data_to_list()

In [84]:
print(encoded_data)

                            file_name       name  label
0      Abomasnow/dcedzyqfojskcahp.jpg  Abomasnow      9
1      Abomasnow/gqfpsmqasdqiknur.png  Abomasnow      9
2      Abomasnow/imzcvkkckbdchpro.png  Abomasnow      9
3      Abomasnow/kzibfmivzksykiwy.png  Abomasnow      9
4      Abomasnow/mjtasvyoonxyilqt.png  Abomasnow      9
...                               ...        ...    ...
12091   Zweilous/nrpzbrzmxehydoqj.png   Zweilous      1
12092   Zweilous/qjoppeepmpyujyao.png   Zweilous      1
12093   Zweilous/sihxufnlbmephyeq.png   Zweilous      1
12094   Zweilous/vshewhewmkutsdlp.png   Zweilous      1
12095   Zweilous/ytpdigaymlnyrpbd.png   Zweilous      1

[12096 rows x 3 columns]


In [85]:
class CustomDataset(Dataset):
	def __init__(self, x, y, img_dir):
		self.x = x
		self.y = y
		self.img_dir = img_dir
		self.classes = np.unique(self.y)


	def __len__(self):
		return len(self.x)


	def __getitem__(self, idx):
		img_path = os.path.join(self.img_dir, self.x[idx])
        # use the slice to remove a possible 4th alpha channel
		image = iio.v2.imread(img_path)[:,:,:3]
		label = self.y[idx]
		return image, label

In [86]:
def stratified_split(dataset):
    x_train, x_val, y_train, y_val = train_test_split(dataset['file_name'].to_numpy(),
                                                  dataset['label'].to_numpy(),
                                                  test_size=0.25,
                                                  stratify=dataset['label'],
                                                  random_state=SEED)

    train = CustomDataset(x_train, y_train, NORMALIZED_DATA)
    val = CustomDataset(x_val, y_val, NORMALIZED_DATA)
    return train, val

In [87]:
dataset = compile_training_data_to_list()
train, val = stratified_split(dataset)

In [88]:
datamodule = ImageClassificationData.from_datasets(train_dataset=train,
                                                   val_dataset=val,
                                                   batch_size=BATCH_SIZE,
                                                   )
performance_metrics = [torchmetrics.Accuracy(),
                      torchmetrics.F1Score(num_classes=len(train.classes), average='macro')]

In [89]:
model = ImageClassifier(backbone='efficientnet_b0',
                        labels=train.classes,
                        metrics=performance_metrics,
                        loss_fn=LabelSmoothingCrossEntropy(0.02),
                        optimizer="AdamW",
                        learning_rate=INIT_LR, )

logger = CSVLogger(save_dir='logs/', version='30Epoch-NoFreeze')

trainer = flash.Trainer(max_epochs=EPOCHS,
                        gpus=torch.cuda.device_count(),
                        logger=logger)

Using 'efficientnet_b0' provided by rwightman/pytorch-image-models (https://github.com/rwightman/pytorch-image-models).
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [90]:
startTime = time.time()
trainer.finetune(model,
                 datamodule=datamodule,
                 strategy='no_freeze')

endTime = time.time()
print(f"[INFO] total time taken to train the model: {(endTime - startTime) / 60 :.2f}min")

C:\Users\CYBORGX\anaconda3\envs\birdclef\lib\site-packages\flash\core\trainer.py:213: UserWarning: The model contains a default finetune callback.
  warnings.warn("The model contains a default finetune callback.", UserWarning)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type           | Params
-------------------------------------------------
0 | loss_fn       | ModuleDict     | 0     
1 | train_metrics | ModuleDict     | 0     
2 | val_metrics   | ModuleDict     | 0     
3 | test_metrics  | ModuleDict     | 0     
4 | adapter       | DefaultAdapter | 4.0 M 
-------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.122    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\CYBORGX\anaconda3\envs\birdclef\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\CYBORGX\anaconda3\envs\birdclef\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

[INFO] total time taken to train the model: 2.59min


C:\Users\CYBORGX\anaconda3\envs\birdclef\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [91]:
trainer.save_checkpoint("saved-models/B0-Un.pt")

In [92]:
metrics = pd.read_csv(f'{trainer.logger.log_dir}/metrics.csv')
display(metrics)
del metrics["epoch"]
metrics.set_index("step", inplace=True)

,train_accuracy_step,train_f1score_step,train_labelsmoothingcrossentropy_step,epoch,step,val_accuracy,val_f1score,val_labelsmoothingcrossentropy,train_accuracy_epoch,train_f1score_epoch,train_labelsmoothingcrossentropy_epoch
0,0.265625,0.133850,2.457307,0,49,NaN,NaN,NaN,NaN,NaN,NaN
1,0.343750,0.243730,2.180583,0,99,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,0,140,0.102513,0.036456,2.768052,NaN,NaN,NaN
3,NaN,NaN,NaN,0,140,NaN,NaN,NaN,0.298205,0.21718,2.29969
4,0.484375,0.380005,1.696713,1,149,NaN,NaN,NaN,NaN,NaN,NaN
5,0.453125,0.387222,1.924132,1,199,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
le = LabelEncoder()
labels = le.fit_transform(filtered_metadata['type1'])
filtered_metadata['encoded_label'] = labels

type_list = filtered_metadata['type1'].unique()
enc_type_list = filtered_metadata['encoded_label'].unique()

type_lookup = pd.DataFrame(list(zip(type_list, enc_type_list)), columns=['Type_name', 'encoded_type'])

<bound method NDFrame.head of            name   type1  encoded_label
0     Bulbasaur   grass              9
1       Ivysaur   grass              9
2      Venusaur   grass              9
3    Charmander    fire              6
4    Charmeleon    fire              6
..          ...     ...            ...
714     Noivern  flying              7
715     Xerneas   fairy              4
716     Yveltal    dark              1
718     Diancie    rock             15
720   Volcanion    fire              6

[703 rows x 3 columns]>
   Type_name  encoded_type
0      grass             9
1       fire             6
2      water            17
3        bug             0
4     normal            12
5     poison            13
6   electric             3
7     ground            10
8      fairy             4
9   fighting             5
10   psychic            14
11      rock            15
12     ghost             8
13       ice            11
14    dragon             2
15      dark             1
16     steel      

In [120]:
pred_data = ImageClassificationData.from_files(
    predict_files=[
        '../../Pokemon-data/Arceus/efplhfkrupqxdmke.png',
    ],
    batch_size = 1
)

predictions = trainer.predict(model, datamodule=pred_data, output="labels")

print(f"""
- Pokemon had the predicted type:
{type_lookup.loc[type_lookup['encoded_type'] == predictions[0][0]]}
---------------------------------------------------------------------
- The Pokemon in Question actually has the type:
{filtered_metadata[filtered_metadata['name'] == 'Arceus']}
""")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\CYBORGX\anaconda3\envs\birdclef\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 96it [00:00, ?it/s]


- Pokemon had the predicted type:
  Type_name  encoded_type
4    normal            12
---------------------------------------------------------------------
- The Pokemon in Question actually has the type:
       name   type1  encoded_label
492  Arceus  normal             12

